In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import h5py
import json
import matplotlib.pyplot as plt
import sparse
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn, Tensor
import numpy as np
from typing import *
from pathlib import Path

In [2]:
PATH = Path("/home/centos/data/cmu_challenge/")

## Neural Network with low level features

In [14]:
class MuonData(Dataset):
    def __init__(self, file_name):
        self.file_name = file_name
        with h5py.File(self.file_name, "r") as file:
            self.len = len(file['fold_0/targets'])
        
    def __len__(self) -> int:
        return self.len
    
    def __del__(self) -> None:
        if hasattr(self, 'file'): self.file.close()
            
    def open_file(self) -> None:
        self.file = h5py.File(self.file_name, "r")
        self.shape = json.loads(self.file['meta_data/matrix_feats'][()])['shape']
        all_hits = self.file['fold_0/matrix_inputs']
        coords = all_hits[1:].astype(int)
        self.hits = sparse.COO(coords=coords, data=all_hits[0], shape=[coords[0][-1]+1]+self.shape)
        self.hl_inputs = self.file['fold_0/inputs'][()]
        self.targets = self.file['fold_0/targets'][()]
    
    def __getitem__(self, idx) -> Tuple[np.ndarray, np.ndarray, float]: 
        if not hasattr(self, 'file'): self.open_file()
        return self.hl_inputs[idx], self.hits[idx].todense(), self.targets[idx]

In [15]:
training_data = MuonData(PATH/'muon_calo_train.hdf5')

In [16]:
len(training_data)

88675

In [6]:
%%time
_ = training_data[0]

CPU times: user 26.9 s, sys: 11.9 s, total: 38.9 s
Wall time: 38.7 s


In [7]:
for i, (hits, hl_feats, labels) in enumerate(DataLoader(training_data, batch_size=16, shuffle=True)):
    print(hits.shape, hl_feats.shape, labels.shape)
    if i == 2: break

torch.Size([16, 28]) torch.Size([16, 1, 50, 32, 32]) torch.Size([16])
torch.Size([16, 28]) torch.Size([16, 1, 50, 32, 32]) torch.Size([16])
torch.Size([16, 28]) torch.Size([16, 1, 50, 32, 32]) torch.Size([16])


In [8]:
class cnn3d(nn.Module):
    def __init__(self, n_layers_per_res:int=1, channel_coef:float=1.5, device:Optional[torch.device]=None):
        super().__init__()
        if device is None: device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.device = device
        self.conv_layers = self._build_conv_layers(n_layers_per_res, channel_coef).to(self.device)
        self.n_conv_out = self._compute_conv_out()
        self.fc_layers = self._build_fc_layers().to(self.device)

    def flatten_x(self, x:Tensor) -> Tensor:
        return x.view(x.size(0),-1)  # flatten tensor to (N, M), could be replaced by a pooling layer

    def _compute_conv_out(self) -> int:
        with torch.no_grad():
            self.conv_layers.eval()
            x = torch.zeros((1,1,50,32,32), device=self.device)
            x = self.conv_layers(x)
            x = self.flatten_x(x)
        self.conv_layers.train()
        return x.shape[1]  # number of features per muon

    def _build_conv_layers(self, n_layers_per_res:int, channel_coef:float) -> nn.Sequential:
        layers = []
        n_in = 1
        kernel_sz = 3

        for i in range(4):
            for j in range(n_layers_per_res):
                if j == 0:
                    # Downsample
                    stride = 2
                    if i == 0:
                        n_out = 8  #  large upscale on first downsample
                    else:
                        n_out = int(n_in*channel_coef)
                else:
                    stride = 1
                    n_out = n_in
                layers.append(self._get_conv_layer(n_in=n_in, n_out=n_out, kernel_sz=kernel_sz, stride=stride))
                n_in = n_out
        return nn.Sequential(*layers)
    
    def _get_conv_layer(self, n_in:int, n_out:int, kernel_sz:Union[int,Tuple[int,int,int]],
                        stride:int=1, padding:Union[str,int,Tuple[int,int,int]]='auto') -> nn.Sequential:
        layers = []
        
        if padding == 'auto': padding = kernel_sz//2 if isinstance(kernel_sz, int) else [i//2 for i in kernel_sz]
        layers.append(nn.Conv3d(in_channels=n_in, out_channels=n_out, kernel_size=kernel_sz,
                                padding=padding, stride=stride, bias=False))
        nn.init.kaiming_normal_(layers[-1].weight, nonlinearity='relu', a=0)
        layers.append(nn.ReLU())
        return nn.Sequential(*layers)

    def _get_fc_layer(self, n_in:int, n_out:int, act:bool=True) -> nn.Sequential:
        layers = []
        layers.append(nn.Linear(n_in, n_out))
        nn.init.kaiming_normal_(layers[-1].weight, nonlinearity='relu', a=0)
        nn.init.zeros_(layers[-1].bias)
        if act: layers.append(nn.ReLU())
        return nn.Sequential(*layers)

    def _build_fc_layers(self) -> nn.Sequential:
        layers = []
        n_in = self.n_conv_out
        for i in range(2):
            n_out = np.max((1,n_in//2))
            layers.append(self._get_fc_layer(n_in, n_out))
            n_in = n_out
        layers.append(self._get_fc_layer(n_in=n_in, n_out=1, act=False))  # Final layer single output, linear activation
        return nn.Sequential(*layers)

    def forward(self, x:Tensor) -> Tensor:
        x = self.conv_layers(x)
        x = self.flatten_x(x)
        x = self.fc_layers(x)
        return x

In [9]:
net = cnn3d()

In [10]:
net

cnn3d(
  (conv_layers): Sequential(
    (0): Sequential(
      (0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv3d(8, 12, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv3d(12, 18, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Conv3d(18, 27, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): ReLU()
    )
  )
  (fc_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=432, out_features=216, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=216, out_features=108, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=108, out_features=1, bias=True)
    )
  )
)

In [18]:
%%time
train_dataloader = DataLoader(training_data, batch_size=256, shuffle=True)
for i, b in enumerate(train_dataloader):
    net(b[1].to(net.device))
    if i == 10: break

CPU times: user 1min 6s, sys: 2.29 s, total: 1min 9s
Wall time: 1.95 s


In [12]:
%%time
train_dataloader = DataLoader(training_data, batch_size=256, shuffle=True, num_workers=2)
for i, b in enumerate(train_dataloader):
    net(b[1].to(net.device))
    if i == 10: break

CPU times: user 6.55 s, sys: 557 ms, total: 7.11 s
Wall time: 1.4 s


In [13]:
%%time
train_dataloader = DataLoader(training_data, batch_size=256, shuffle=True, num_workers=4)
for i, b in enumerate(train_dataloader):
    net(b[1].to(net.device))
    if i == 10: break

CPU times: user 6.43 s, sys: 637 ms, total: 7.07 s
Wall time: 1.06 s
